# Video pipeline with Yolo + Resnet Model

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import pandas as pd
import sys
import time
from typing import List
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)


# get an absolute path to the directory that contains parent files
project_dir = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(
    project_dir, '..', '..')))

from experiments.utils.constants import (
    PIPELINE_SIMULATION_CONFIGS_PATH,
    PIPELINE_SIMULATION_RESULTS_PATH
)

series = 4
series_path = os.path.join(
    PIPELINE_SIMULATION_RESULTS_PATH, 'series', str(series))

In [2]:
all_states = pd.read_csv(
    os.path.join(series_path, 'all-states.csv'))
with_constraints = pd.read_csv(
    os.path.join(series_path, 'with-constraints.csv'))
optimal = pd.read_csv(
    os.path.join(series_path, 'optimal.csv'))
all_states.columns

Index(['task_0_variant', 'task_0_cpu', 'task_0_gpu', 'task_0_cpu_all_replicas',
       'task_0_gpu_all_replicas', 'task_0_batch', 'task_0_replicas',
       'task_0_latency', 'task_0_throughput', 'task_0_throughput_all_replicas',
       'task_0_measured', 'task_1_variant', 'task_1_cpu', 'task_1_gpu',
       'task_1_cpu_all_replicas', 'task_1_gpu_all_replicas', 'task_1_batch',
       'task_1_replicas', 'task_1_latency', 'task_1_throughput',
       'task_1_throughput_all_replicas', 'task_1_measured', 'pipeline_latency',
       'pipeline_throughput', 'pipeline_cpu', 'pipeline_gpu',
       'accuracy_objective', 'alpha', 'beta', 'objective', 'pipeline_accuracy',
       'resource_objective', 'task_0_accuracy', 'task_1_accuracy'],
      dtype='object')

In [3]:
all_states

,task_0_variant,task_0_cpu,task_0_gpu,task_0_cpu_all_replicas,task_0_gpu_all_replicas,task_0_batch,task_0_replicas,task_0_latency,task_0_throughput,task_0_throughput_all_replicas,...,pipeline_cpu,pipeline_gpu,accuracy_objective,alpha,beta,objective,pipeline_accuracy,resource_objective,task_0_accuracy,task_1_accuracy
0,yolov5n,1,0,1,0,1,1,0.093966,10.642173,10.642173,...,2,0,0.25,1.0,1.0,0.7500,0.25,0.5000,0.5,0.5
1,yolov5n,1,0,1,0,1,1,0.093966,10.642173,10.642173,...,2,0,0.25,1.0,1.0,0.7500,0.25,0.5000,0.5,0.5
2,yolov5n,1,0,1,0,1,1,0.093966,10.642173,10.642173,...,2,0,0.25,1.0,1.0,0.7500,0.25,0.5000,0.5,0.5
3,yolov5n,1,0,1,0,1,1,0.093966,10.642173,10.642173,...,2,0,0.25,1.0,1.0,0.7500,0.25,0.5000,0.5,0.5
4,yolov5n,1,0,1,0,1,1,0.093966,10.642173,10.642173,...,2,0,0.25,1.0,1.0,0.7500,0.25,0.5000,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36859,yolov5m,4,0,8,0,16,2,0.290719,55.035929,110.071859,...,16,0,0.25,1.0,1.0,0.3125,0.25,0.0625,0.5,0.5
36860,yolov5m,4,0,8,0,16,2,0.290719,55.035929,110.071859,...,16,0,0.25,1.0,1.0,0.3125,0.25,0.0625,0.5,0.5
36861,yolov5m,4,0,8,0,16,2,0.290719,55.035929,110.071859,...,16,0,0.25,1.0,1.0,0.3125,0.25,0.0625,0.5,0.5
36862,yolov5m,4,0,8,0,16,2,0.290719,55.035929,110.071859,...,16,0,0.25,1.0,1.0,0.3125,0.25,0.0625,0.5,0.5
